Original source code and data found here: https://github.com/FlorentBuissonOReilly/BehavioralDataAnalysis

In [1]:
import pandas as pd
hist_data_df = pd.read_csv('data/chap8-historical_data.csv')
exp_data_df = pd.read_csv('data/chap8-experimental_data.csv')

In [2]:
effect = 0.01
historical_conversion_rate = 0.184
new_conversion_rate = historical_conversion_rate + effect
new_conversion_rate

0.194

In [3]:
# 1% effect is a 5.4% increase
(new_conversion_rate - historical_conversion_rate) / historical_conversion_rate

0.05434782608695657

In [4]:
import statsmodels.stats.proportion as ssprop # To calculate the standardized effect size
effect_size = ssprop.proportion_effectsize(new_conversion_rate, historical_conversion_rate)
effect_size

0.02554423106645265

I'm not sure exactly what effect_size is doing, despite knowing the algorithm

`2 * (arcsin(sqrt(prop1)) - arcsin(sqrt(prop2)))`

In [9]:
# calculate sample size to detect 1%
# kindle location 5430
# "As long as our treatment doesn't increase our booking rate,
# we don't really care whether it has the same booking rate or a lower booking rate compared to our control;
# either way we won't implement it. This implies that we can run a one-sided test instead of a two-sided test ..."
import statsmodels.stats.power as ssp # To calculate the standard power
from math import ceil

per_group_size = ssp.tt_ind_solve_power(
    effect_size = effect_size, 
    alpha = 0.05, 
    nobs1 = None, 
    alternative = 'larger', 
    power=0.8
)
print(per_group_size)
print(ceil(per_group_size) * 2)

18950.818821440742
37902


In [18]:
def sample_size(
        conversion_rate: float,
        effect: float=0.01,
        alpha: float=0.05,
        power: float = 0.8) -> int:
    """
    Args:
        conversion_rate:
            the original/historical conversion rate of interest
        effect:
            the assumed effect size in percentage points (not percent change)
            e.g. 0.01 is 1 percentage point e.g. going from conversion rate of 10% to 11%
        alpha:
            statistical significance i.e. false positive rate
            
            How often would we declare success when there is exactly no impact? A value of `0.05`
            means we would implement the treatment %5 of the time (when there is exactly no
            impact). When there is actually an underlying negative affect, it would be less than 5%
            of the time.
            - See table in Behavior Data Analysis loc. 5367
        power:
            If there is actually positive effect of the treatment by exactly `effect` percentage
            points, how often would we detect the positive effect and implement the treatment?
            A value of `0.8` means we would detect the change and implement the treatment 80% of
            the time when there is exactly exactly an `effect` percentage points increase.
    """
    import statsmodels.stats.proportion as ssprop # To calculate the standardized effect size
    import statsmodels.stats.power as ssp # To calculate the standard power
    from math import ceil

    effect_size = ssprop.proportion_effectsize(conversion_rate + effect, conversion_rate)
    per_group_size = ssp.tt_ind_solve_power(
        effect_size=effect_size, 
        alpha=alpha, 
        nobs1=None, 
        alternative='larger', 
        power=power
    )
    
    return ceil(per_group_size) * 2

In [48]:
print(sample_size(conversion_rate=0.184, effect=0.01))
print(sample_size(conversion_rate=0.184, effect=0.01) / 2)


37902
18951.0


In [47]:
import numpy as np
import plotly_express as px
from itertools import product

combos = pd.DataFrame(
    product(np.arange(0.8, 0.96, 0.01), [0.1, 0.05, 0.01]),
    columns=['power', 'alpha']
)
combos['Sample Size'] = combos.apply(
    lambda x: sample_size(conversion_rate=0.184, effect=0.01, alpha=x['alpha'], power=x['power']),
    axis=1
)
fig = px.line(
    data_frame=combos,
    x='power',
    y='Sample Size',
    markers=True,
    color='alpha',
    title="Sample Size Required for different values of `power` and `alpha`"
)
fig.update_layout(yaxis={'range':[0, max(combos['Sample Size']) * 1.10]})

---

In [64]:
len(hist_data_df)

319757

In [49]:
hist_data_df.head()

,age,gender,period,seasonality,month,booked
0,34,male,1,0.749993,2,0
1,39,male,1,0.749993,2,0
2,42,male,1,0.749993,2,0
3,33,male,1,0.749993,2,0
4,32,male,1,0.749993,2,1


In [65]:
len(exp_data_df)

40160

In [50]:
exp_data_df.head()

,age,gender,period,seasonality,month,booked,oneclick
0,47,male,33,6.492170e-08,10,0,1
1,49,male,33,6.492170e-08,10,0,1
2,37,male,33,6.492170e-08,10,0,0
3,44,male,33,6.492170e-08,10,0,1
4,45,male,33,6.492170e-08,10,0,1


In [63]:
### Basic random assignment
K = 2
assgnt = np.random.uniform(0,1,1)
group = "control" if assgnt <= 1/K else "treatment"

print(assgnt)
print(group)

[0.61370824]
treatment


In [66]:
np.where(np.random.uniform(0,1,2000) > 0.5, 1, 0)

array([1, 1, 0, ..., 1, 1, 0])

In [346]:
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### Null experimental dataset
exp_null_data_df = hist_data_df.copy().sample(20000)
exp_null_data_df['oneclick'] = np.where(np.random.uniform(0,1,20000) >= 0.5, 1, 0)
mod = smf.logit('booked ~ oneclick + age + gender', data = exp_null_data_df)
mod.fit(disp=0).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                20000
Model:                          Logit   Df Residuals:                    19996
Method:                           MLE   Df Model:                            3
Date:                Sun, 18 Sep 2022   Pseudo R-squ.:                  0.2788
Time:                        11:31:06   Log-Likelihood:                -6929.9
converged:                       True   LL-Null:                       -9608.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.2950      0.203     50.766      0.000       9.897      10.692
gender[T.male]     0.0818      0.043      1.898      0.058      -0.003       0.166
oneclick          -0.0082      0.043     -0.191      0.848      -0.093       0.076
age               -0.3179      0.006    -56.941      0.000      -0.329      -0.307
==================================================================================
"""

In [348]:
exp_null_data_df.groupby('oneclick')['booked'].agg(np.mean)

oneclick
0    0.190208
1    0.181800
Name: booked, dtype: float64

In [349]:
## Metric function
def log_reg_fun(dat_df):
    model = smf.logit('booked ~ oneclick + age + gender', data = dat_df)
    res = model.fit(disp=0)
    coeff = res.params['oneclick']
    return coeff

log_reg_fun(exp_null_data_df)

-0.008228921245909264

In [350]:
## Bootstrap CI function
def boot_CI_fun_A(dat_df, metric_fun, B = 100, conf_level = 0.9):
    #Setting sample size
    N = len(dat_df)
    conf_level = conf_level
    coeffs = []

    for i in range(B):
        sim_data_df = dat_df.sample(n=N, replace = True)
        coeff = metric_fun(sim_data_df)
        coeffs.append(coeff)

    return coeffs

coeffs_result = boot_CI_fun_A(dat_df=exp_null_data_df, metric_fun=log_reg_fun, B=10000)
coeffs_result[0:4]

px.histogram(pd.DataFrame({'x': coeffs_result}), x='x')

In [352]:
(np.array(coeffs_result) > 0).mean()

0.4225

In [353]:
## Bootstrap CI function
def boot_CI_fun(dat_df, metric_fun, num_sims = 100, conf_level = 0.9):
    """Create bootstrap confidence intervals for a given number of simulations/resamples."""
    num_records = len(dat_df)
    
    coeffs = [metric_fun(dat_df.sample(n=num_records, replace = True)) for _ in range(num_sims)]
    coeffs.sort()
    start_idx = round(num_sims * (1 - conf_level) / 2)
    end_idx = -start_idx
    confint = [coeffs[start_idx], coeffs[end_idx]]  
    return(confint)
    

boot_CI_fun(dat_df=exp_null_data_df, metric_fun=log_reg_fun, num_sims=1000)

[-0.08010040355297313, 0.062160161218775205]

In [201]:
boot_CI_fun(dat_df=exp_null_data_df, metric_fun=log_reg_fun, num_sims=1000)

[-0.0907309354332875, 0.4012764423164247]

In [204]:
boot_CI_fun(dat_df=exp_null_data_df, metric_fun=log_reg_fun, num_sims=10000)

[-0.07873427176239532, 0.3782230769491403]

In [205]:
boot_CI_fun(dat_df=exp_null_data_df, metric_fun=log_reg_fun, num_sims=10000)

[-0.07791478865064418, 0.37902062686888716]

In [149]:


## decision function
def decision_fun(dat_df, metric_fun, num_sims = 100, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, num_sims = num_sims, conf_level = conf_level)
    decision = 1 if boot_CI[0] > 0  else 0
    return decision 

## Function for single simulation
def single_sim_fun(Nexp, dat_df = hist_data_df, metric_fun = log_reg_fun, 
                   eff_size = 0.01, B = 100, conf_level = 0.9):
    
    #Adding predicted probability of booking
    hist_model = smf.logit('booked ~ age + gender + period', data = dat_df)
    res = hist_model.fit(disp=0)
    sim_data_df = dat_df.copy()
    sim_data_df['pred_prob_bkg'] = res.predict()
    #Filtering down to desired sample size
    sim_data_df = sim_data_df.sample(Nexp)
    #Random assignment of experimental groups
    sim_data_df['oneclick'] = np.where(np.random.uniform(size=Nexp) <= 0.5, 0, 1)
    # Adding effect to treatment group
    sim_data_df['pred_prob_bkg'] = np.where(sim_data_df.oneclick == 1, 
                                            sim_data_df.pred_prob_bkg + eff_size, 
                                            sim_data_df.pred_prob_bkg)
    sim_data_df['booked'] = np.where(sim_data_df.pred_prob_bkg >= \
                                     np.random.uniform(size=Nexp), 1, 0)
    
    #Calculate the decision (we want it to be 1)
    decision = decision_fun(sim_data_df, metric_fun = metric_fun, num_sims = B, 
                            conf_level = conf_level)
     
    return decision  
 
## power simulation function
def power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100, 
                  conf_level = 0.9):
    power_lst = []
    for i in range(Nsim):
        power_lst.append(single_sim_fun(Nexp = Nexp, dat_df = dat_df, 
                                        metric_fun = metric_fun, 
                                        eff_size = eff_size, B = B, 
                                        conf_level = conf_level))
    power = np.mean(power_lst)
    return(power)

In [206]:
len(hist_data_df)

319757

In [156]:
## Single simulation
[single_sim_fun(Nexp = 1000) for _ in range(20)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [157]:
int(4e4)

40000

In [158]:
## Power simulation
power_sim_fun(dat_df=hist_data_df, metric_fun = log_reg_fun, Nexp = int(4e4), 
              eff_size=0.01, Nsim=20)

0.95

---

In [ ]:
px.histogram(pd.DataFrame({'x': coeffs_result}), x='x')

In [240]:
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### Null experimental dataset
exp_null_data_df = hist_data_df.copy().sample(20000)
exp_null_data_df['oneclick'] = np.where(np.random.uniform(0,1,20000) > 0.5, 1, 0)
mod = smf.logit('booked ~ oneclick + age + gender', data = exp_null_data_df)
mod.fit(disp=0).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                20000
Model:                          Logit   Df Residuals:                    19996
Method:                           MLE   Df Model:                            3
Date:                Sun, 18 Sep 2022   Pseudo R-squ.:                  0.2751
Time:                        10:51:40   Log-Likelihood:                -6822.0
converged:                       True   LL-Null:                       -9411.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.2645      0.205     49.950      0.000       9.862      10.667
gender[T.male]     0.0382      0.043      0.879      0.379      -0.047       0.123
oneclick           0.0658      0.043      1.517      0.129      -0.019       0.151
age               -0.3185      0.006    -56.204      0.000      -0.330      -0.307
==================================================================================
"""

In [241]:
len(exp_null_data_df)

20000

In [354]:
## Bootstrap CI function
from typing import Callable, Iterable

def bootstrap_metric(
        data: pd.DataFrame,
        metric_func: Callable[[pd.DataFrame], float],
        num_reps: int = 100,
        conf_level: float = 0.9) -> Iterable[float]:
    """Create bootstrap samples of the metric generated from `metric_func`"""
    num_records = len(data)
    return np.array([metric_func(data.sample(n=num_records, replace = True)) for _ in range(num_reps)])


metrics = bootstrap_metric(data=exp_null_data_df, metric_func=log_reg_fun, num_reps=1000)
px.histogram(pd.DataFrame({'x': metrics}), x='x')

In [355]:
(metrics > 0).mean()

0.418